# Expliquer le vote populiste en France

## Première partie : Le vote comme expression d'opinions subjectives

« L'isoloir planté dans une salle d'école ou de mairie est le symbole de toutes les trahisons que l'individu peut commettre envers les groupes dont il fait partie. ». Cette phrase suffirait presque à résumer l'argumentaire de Sartre dans son célèbre texte "Élections, piège à cons", publié dans la revue Les Temps Modernes en 1973. Pour Sartre, le fait que la pratique du vote soit dans nos démocraties modernes un acte individuel empêche les classes sociales de porter collectivement leur intérêt, car chacun peut se défiler au moment de prendre la décision importante qui est celle d'élire les réprésentants du peuple. En effet, le ressenti individuel du votant peut ne pas correspondre à l'analyse objective que ferait un observateur extérieur de ce que devrait chosir le votant en fonction de ses caractéristiques socio-économiques objectives (niveau d'éducation, revenu,... en bref sa classe sociale). Or, c'est bien ce ressenti individuel et subjectif qui détermine en dernière instance le vote. Les caractéristiques individuelles objectives ne sont que des facteurs explicatifs parmi d'autres de ce ressenti. C'est pourquoi la sociologie électorale ne se contente pas de lier l'environnement social des individus à leur vote mais cherche également à collecter leur ressenti subjectif sur le monde dans lequel ils vivent.

La base de données à notre disposition permet justement de prendre en compte ce ressenti subjectif. En effet, la plupart des variables sont des réponses à des questions d'opinion. Le but de cette partie est de lier ce ressenti subjectif aux déclarations de vote (en particulier du vote FN) des individus. Devant l'étendue des variables et thématiques disponibles, nous avons construit 5 variables subjectives synthétiques nous semblant pertinentes pour expliquer le vote FN.

Avant d'expliquer comment nous construisons ces variables, on peut dans un premier temps décrire brièvement la base en question à l'aide de quelques statistiques descriptives.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels as sm
import statsmodels.api as st
import statsmodels.formula.api as smf
import urllib
import bs4

In [8]:
base_indiv = pd.read_spss('base_indiv.sav', convert_categoricals=False)
base_indiv.shape

(2070, 539)

Comme on peut le voir, la base est extrêmement large. À l'aide du descriptif de chaque variable, que l'on peut trouver en ligne (http://nesstar.ess.nsd.uib.no/webview/index.jsp?v=2&submode=abstract&study=http%3A%2F%2F129.177.90.83%3A80%2Fobj%2FfStudy%2FESS9e02.0&mode=documentation&top=yes), nous avons présélectionné un sous-ensemble de 37 variables, qui recouvrent à notre sens l'essentiel des variables subjectives et objectives pertinentes pour notre problématique. Nous avons également veillé à ne pas chosir des variables redondantes. La description de chaque variable est disponible dans un document annexe (README).

In [9]:
base_indiv = base_indiv[["ess8_id", "nuts2", "ppltrst", "cptppola", "trstplt", "ccnthum", "polintr", "vote", "pbldmn","bctprd","pstplonl", "mbtru", "stflife", "stfedu", "stfhlth", "aesfdrk", "health",  "gincdif", "euftf", "imwbcnt", "ipstrgv", "mnrgtjb", "freehms", "imptrad", "rlgblg", "blgetmg", "crmvct", "prtvtcfr", "gndr", "agea", "chldhm", "edlvdfr","uemp3m", "tporgwk", "hinctnta", "lrscale", "pspwght"]]
base_indiv = base_indiv.rename(columns={'pspwght' : 'weight'})
base_indiv.describe()

,ppltrst,cptppola,trstplt,ccnthum,polintr,vote,pbldmn,bctprd,pstplonl,mbtru,...,prtvtcfr,gndr,agea,chldhm,edlvdfr,uemp3m,tporgwk,hinctnta,lrscale,weight
count,2070.000000,2055.000000,2059.000000,2024.000000,2069.000000,2003.00000,2068.000000,2065.000000,2068.000000,2063.000000,...,1059.000000,2070.000000,2069.000000,2070.000000,2068.000000,2066.000000,1936.000000,1884.000000,1916.000000,2070.000000
mean,4.583575,2.195134,2.893152,3.614625,2.559691,1.52671,1.856867,1.676513,1.803191,2.713524,...,8.600567,1.539614,52.377960,1.701449,10.130561,1.610842,3.620351,4.855626,5.079854,1.000000
std,2.143143,0.943060,2.094522,2.907148,0.970024,0.70722,0.350294,0.467920,0.397683,0.591605,...,3.167957,0.498549,18.929428,0.457733,6.134469,0.487677,1.176417,2.740380,2.409379,0.691451
min,0.000000,1.000000,0.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,15.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.228504
25%,3.000000,1.000000,1.000000,3.000000,2.000000,1.00000,2.000000,1.000000,2.000000,3.000000,...,9.000000,1.000000,38.000000,1.000000,6.000000,1.000000,3.000000,2.000000,3.000000,0.539700
50%,5.000000,2.000000,3.000000,3.000000,3.000000,1.00000,2.000000,2.000000,2.000000,3.000000,...,9.000000,2.000000,53.000000,2.000000,7.000000,2.000000,4.000000,5.000000,5.000000,0.803473
75%,6.000000,3.000000,4.000000,4.000000,3.000000,2.00000,2.000000,2.000000,2.000000,3.000000,...,10.000000,2.000000,67.000000,2.000000,15.000000,2.000000,4.000000,7.000000,7.000000,1.203349
max,10.000000,5.000000,10.000000,55.000000,4.000000,3.00000,2.000000,2.000000,2.000000,3.000000,...,16.000000,2.000000,99.000000,2.000000,26.000000,2.000000,6.000000,10.000000,10.000000,4.000000


On peut faire apparaître les valeurs manquantes de façon plus lisible.

In [10]:
base_indiv.isnull().sum()

ess8_id        0
nuts2          0
ppltrst        0
cptppola      15
trstplt       11
ccnthum       46
polintr        1
vote          67
pbldmn         2
bctprd         5
pstplonl       2
mbtru          7
stflife        2
stfedu        20
stfhlth        5
aesfdrk       48
health         2
gincdif       14
euftf         81
imwbcnt       27
ipstrgv       21
mnrgtjb        6
freehms       19
imptrad       14
rlgblg         8
blgetmg       39
crmvct         2
prtvtcfr    1011
gndr           0
agea           1
chldhm         0
edlvdfr        2
uemp3m         4
tporgwk      134
hinctnta     186
lrscale      154
weight         0
dtype: int64

On s'aperçoit que seule la variable prtvtcfr (parti choisi aux dernières élections nationales) a plus de 10% de valeurs manquantes. Comment expliquer cette lacune pour cette variable ? La variable vote (qui prend la valeur 1 quand l'individu a voté aux dernières élections, 2 quand il n'a pas voté et 3 quand il ne pouvait pas voter) donne un premier élément de réponse à cette question.

In [14]:
base_indiv['vote'].value_counts()

1.0    1199
2.0     553
3.0     251
Name: vote, dtype: int64

In [21]:
base_indiv['vote'][base_indiv['vote']==1].count() - base_indiv[base_indiv['prtvtcfr'].notnull()]['prtvtcfr'].count()

140

In [19]:
base_indiv['prtvtcfr'].notnull().count()

2070

Les valeurs manquantes s'expliquent donc par les abstentionnistes (553), les non-inscrits sur les listes électorales (251) et ceux n'ayant pas voulu dire pour qui ils avaient voté (140). 

### Proxy confiance

### Modèle avec variables subjectives

Maintenant que nous avons créé ces 5 proxys nous semblant pertinent pour expliquer le vote FN, nous allons les tester pour voir comment chacun influe sur la probabilité de voter pour le Front National. Pour ce faire, nous utilisons un modèle logistique binaire, car la variable d'intérêt est une variable de choix binaire (voter ou non pour le FN). Le but est donc d'évaluer la probabilité de voter FN en fonction des variables subjectives que nous avons construites. Or, utiliser dans ce cas un modèle linéaire classique est moins pertinent car cela peut conduire à estimer des probabilités qui sont en-dehors de l'intervalle [0,1]. Nous aurions pu également utiliser le modèle binaire probit (avec un terme d'erreur gaussien et non logistique), mais cela est moins courramment utilisé dans la littérature.

Tout d'abord, nous restreignons nos observations aux individus qui ont voté et qui ont déclaré pour qui ils avaient voté. Nous ne considérons pas les abstentionnistes car l'interprétation de l'abstention est difficile. En effet, celle-ci peut être l'expression d'un choix politique à part entière, ou d'une simple négligence, d'un oubli. L'analyse de l'abstention mériterait donc une analyse à part entière, distincte du vote populiste. En revanche, nous considérons les votes blanc et nul.

Nous créons donc une nouvelles base "base_indiv_vote", puis une variable indicatrice du vote FN (pour FN, prtvtcfr vaut 2).

In [ ]:
base_indiv_vote = base_indiv.dropna(subset = ['prtvtcfr'])
base_indiv_vote['FN'] = np.where(base_indiv_vote['prtvtcfr'] == 2, 1, 0)
base_indiv_vote['FN'].mean()

La proportion de votants FN parmi les votants est proche de 12%, ce qui sous-estime largement le résultats observé de Marine Le Pen au premier tour de l'élection de 2012 (qui est l'élection sur laquelle les individus étaient interrogés), qui était de près de 18%. On peut voir que le problème reste le même lorsque nous corrigeons pour les poids (représentant l'inverse de la probabilité d'être interrogé : plus celle-ci est faible, plus il faut donner de poids à l'individu). 

In [ ]:
base_indiv_vote['FN_weight'] = base_indiv_vote['FN']*base_indiv_vote['weight']
base_indiv_vote['FN_weight'].mean()

Ceci peut être dû à la honte persistante que représente le vote FN en France. Cela justifie que nous délaisserons progressivement dans l'analyse cette variable et que nous nous concentrerons sur les déterminants des variables subjectives motivant le vote FN. Mais avant cela, il importe de montrer la pertinence de ces variables.
Tout d'abord, on peut noter que le vote populiste est mal interprétable selon le schéma classique opposant droite et gauche. Pour le montrer, considérons la variables lrscale, qui est un indicateur (sur une échelle de 0 à 10) de la place que se donne chacun sur l'échiquier politique (0 correspond à la place la plus à gauche et 10 à la place la plus à droite). Ce petit code permet de compter la proportion d'électeurs FN se plaçant sur la moitié gauche de l'échiquier politique.

On voit donc que 35% des électeurs FN se place sur la moitié gauche de l'échiquier politique! À l'inverse, cette proportion tombe à moins de 2% pour les électeurs UMP (pour l'UMP, prtvtcfr vaut 10).

Ceci justifie d'avoir créé des proxy qui sont radicalement différent du clivage droite/gauche.

Nous allons maintenant nous intéresser à la pertinence de chaque proxy pris individuellement dans l'influence du vote FN. Pour chaque proxy, nous représentons graphiquement cet impact selon une spécification logistique. La courbe représente donc la probabilité de voter FN en fonction de la valeur du proxy considéré. Dans chaque cas, un intervalle de confiance à 95% est représenté. Pour rendre la lecture de ces graphiques plus agréable, l'attribut "y_jitter" permet d'introduire un petit aléa à la variable dépendante (FN), qui ne vaut plus exactement 0 ou 1 pour ne pas concentrer les points sur seulement deux valeurs en ordonnées.

In [ ]:
sns.regplot(x = 'confiance', y = 'FN', data = base_indiv_vote, y_jitter = 0.02, logistic = True)

In [ ]:
sns.regplot(x = 'bienêtre', y = 'FN', data = base_indiv_vote, y_jitter = 0.02, logistic = True)

In [ ]:
sns.regplot(x = 'protection', y = 'FN', data = base_indiv_vote, y_jitter = 0.02, logistic = True)

In [ ]:
sns.regplot(x = 'progressisme', y = 'FN', data = base_indiv_vote, y_jitter = 0.02, logistic = True)

Tous les proxy, à l'exception du proxy engagement, semblent significatifs, et plus particulièrement les proxy protection, confiance et bien-être. Nous pouvons maintenant le vérifier formellement par des régressions logistiques. Pour cela, nous utlisons le package statsmodels, nous spécifions la variable dépendante (FN) et la variable explicative (proxy) à laquelle nous ajoutons une constante. Dans chaque cas, nous exposons en sortie les effets marginaux (par la commande "get_margeff"), qui sont les estimations intéressantes à interpréter dans le cas du modèle logit. En effet, l'effet marginal s'interprète comme un coefficient classique dans un modèle linéaire, c'est-à-dire comme la variation en probabilité de voter FN qu'implique la variation d'une unité de la variable proxy considérée. Une autre possibilité est d'interpréter directement le coefficient de la régression logistique en termes de variation de l'odds ratio (= probabilité de voter FN sur la probabilité de ne pas voter FN). Nous proposons ci-dessous les 2 interprétations pour chaque proxy.

In [ ]:
Y = base_indiv_vote['FN']
X = base_indiv_vote['confiance']
X = sm.tools.add_constant(X)
mdl = st.Logit(Y,X)
mdl_margeff = mdl.fit().get_margeff()
print(mdl_margeff.summary())

L'augmentation d'une unité du proxy confiance baisse la probabilité de voter FN de 0.05. 

In [ ]:
print(mdl.fit().summary())

L'augmentation d'une unité du proxy confiance baisse l'odds ratio du vote FN de 50%.

In [ ]:
Y = base_indiv_vote['FN']
X = base_indiv_vote['bienêtre']
X = sm.tools.add_constant(X)
mdl = st.Logit(Y,X)
mdl_margeff = mdl.fit().get_margeff()
print(mdl_margeff.summary())

L'augmentation d'une unité du proxy bien-être baisse la probabilité de voter FN de 0.03. 

In [ ]:
print(mdl.fit().summary())

L'augmentation d'une unité du proxy bien-être baisse l'odds ratio du vote FN de 32%.

In [ ]:
Y = base_indiv_vote['FN']
X = base_indiv_vote['protection']
X = sm.tools.add_constant(X)
mdl = st.Logit(Y,X)
mdl_margeff = mdl.fit().get_margeff()
print(mdl_margeff.summary())

L'augmentation d'une unité du proxy bien-être baisse la probabilité de voter FN de 0.05. 

In [ ]:
print(mdl.fit().summary())

L'augmentation d'une unité du proxy bien-être baisse l'odds ratio du vote FN de 58%.

In [ ]:
Y = base_indiv_vote['FN']
X = base_indiv_vote['progressisme']
X = sm.tools.add_constant(X)
mdl = st.Logit(Y,X)
mdl_margeff = mdl.fit().get_margeff()
print(mdl_margeff.summary())

L'augmentation d'une unité du proxy bien-être baisse la probabilité de voter FN de 0.01. 

In [ ]:
print(mdl.fit().summary())

L'augmentation d'une unité du proxy bien-être baisse l'odds ratio du vote FN de 14%.

On confirme donc la significativité de tous les proxys à l'exception de celui tentant d'approcher le degré d'engagement individuel dans la vie politique et sociale. Cela peut être dû à la grande diversité que peut recouvrir cet engagement.

Nous allons maintenant intégrer les 4 proxys dans une même régression et voir lesquelles restent significatifs.

In [ ]:
Y = base_indiv_vote['FN']
X = base_indiv_vote[['confiance', 'bienêtre', 'protection', 'progressisme']]
X = sm.tools.add_constant(X)
mdl = st.Logit(Y,X)
mdl_margeff = mdl.fit().get_margeff()
print(mdl_margeff.summary())

Les proxys qui ont un effet spécifique (i.e dont l'impact ne peut être complétement absorbé par un autre) sont donc le proxy confiance et le proxy protection. On peut donc en conclure que les deux variables subjectives les plus pertinentes pour expliquer le vote populiste de droite sont, selon notre analyse, le degré de confiance interpersonnelle et envers les institutions (politiques, scientifiques,...) ainsi que le degré de demande de protection vis-à-vis de l'extérieur (d'individus étrangers et d'entité supranationales comme l'Union Européenne). Ces résultats sont robustes à la spécification du modèle retenue, puisque si on effectue une régression linéaire classique on trouve des résultats (en termes de magnitude des effets marginaux et de la significativité) très similaire.

In [ ]:
Y = base_indiv_vote['FN']
X = base_indiv_vote[['confiance', 'bienêtre', 'protection', 'progressisme']]
X = sm.tools.add_constant(X)
mod = st.OLS(Y,X)
print(mod.fit().summary())

## Deuxième partie : Les déterminants objectifs du ressenti individuel

Nous venons donc de voir qu'il était pertinent d'étudier le ressenti subjectif des individus sur des aspects spécifiques comme la confiance et le sentiment d'insécurité vis-à-vis de l'extérieur pour expliquer le vote populiste. Cependant, de telles enquêtes incluant des questions subjectives demeurent rares, et pour généraliser ces résultats il serait bon de lier ce ressenti subjectif à des déterminants objectifs. Cela permet en outre de tirer parti de l'ensemble de la base de données à notre disposition, puisque le taux de réponse aux questions autres que celles concernant directement le vote est bien meilleur. On revient donc à l'ensemble de notre base.
On sélectionne un sous-échantillon de 10 variables objectives, à la fois culturelles, socio-démographiques et socio-économiques : 

-l'appartenance à une religion (qu'on aurait pu classer dans les variables subjectives mais dans la mesure où le milieu culturel d'origine détermine fortement cette variable nous la considérons comme objective) (rlgblg)

-l'appartenance à une minorité ethnique (blgetmg)

-le fait d'avoir un membre de son foyer (y compris soi-même) qui a été victime de harcélement ou d'agression dans les 5 dernières années (crmvct)

-le genre (gndr)

-l'âge (agea)

-le fait de vivre avec ses enfants (chldhm), car cela peut être vecteur de confiance et de bien-être

-le niveau d'éducation (edlvdfr)

-le fait d'avoir connu une période de chômage de plus de 3 mois dans sa vie (uemp3m)

-le type d'organisation pour lequel le répondant travaille (tporgwk)

-la place (en terme d'appartenance à un certain décile) dans l'échelle nationale de niveau de vie (hinctnta) (cette variabe est construite ex-post par les statisticiens d'ESS à partir des réponses des individus en termes monétaires)

On commence par vérifier que les réponses aux variables objectives sont cohérentes et ne contiennent pas de réponse du type "je ne sais pas" ou "je ne souhaite pas répondre". 

In [ ]:
var = base_indiv[['rlgblg', 'blgetmg', 'crmvct', 'gndr', 'agea', 'chldhm', 'edlvdfr', 'uemp3m', 'tporgwk', 'hinctnta']]
for ele in var:
    print(base_indiv[ele].describe())

On voit que les valeurs de toutes les variables s'incrivent dans l'échelle principale (exposée dans le README). De plus, l'analyse des valeurs manquantes faite en début de notebook permet de supprimer les valeurs manquantes pour les variables qui en ont très peu (moins de 2%) sans que cela n'affecte significativement la base.

In [ ]:
base_indiv.dropna(subset = ['rlgblg', 'blgetmg', 'crmvct', 'gndr', 'agea', 'chldhm', 'edlvdfr', 'uemp3m'], inplace = True)

On s'intéresse maintenant plus précisément à la variable du niveau d'éducation individuel.

In [ ]:
base_indiv['edlvdfr'].nunique()

Le niveau d'éducation est trop finement reporté pour pouvoir être analysé. On crée donc une nouvelle variable qui ne prend que 6 niveaux d'éducation : le niveau 0 correpond à un niveau d'étude inférieur ou égal au brevet des collèges, le niveau 1 à un niveau d'études directement professionalisant (CAP ou baccalauréat professionnel), le niveau 2 au baccalauréat (général et technologique), le niveau 3 à l'équivalent d'une licence, le niveau 4 à l'équivalent d'un master et le niveau 5 au doctorat. Pour ce faire, comme les niveaux initiaux sont ordonnés, il suffit de séparer l'échelle en 6 intervalles grâce à la fonction pd.cut(). Nous créons une fonction générale capable de générer cette nouvelle variable, puis nous l'appliquons à notre base.

In [ ]:
def level_education(df):
    level = pd.cut(x=df['edlvdfr'],
            bins=[1, 5, 9, 13, 18, 24, 26],
            include_lowest=True,
            labels=[0, 1, 2, 3, 4, 5]).astype(int)
    df['level_education'] = level

level_education(base_indiv)

On peut maintenant observer la distribution de cette variable dans la population.

In [ ]:
sns.distplot(base_indiv['level_education'], kde = False)

Nous allons maintenant créer deux nouvelles variables indicatrices prenant en compte le type d'organisation pour lequel l'individu interrogé travaille. En effet, les fonctionnaires jouissent d'un statut qui leur assure souvent une protection supplémentaire contre certains risques (sécurité de l'emploi, retraite meilleure, meilleure assurance,...), ce qui peut influer sur la confiance, le bien-être et la demande de protection. À l'inverse, les indépendants sont soumis à plus d'aléas, ce qui peut également se repercuter sur leur vision du monde (souvent plus conservatrice voire plus réactionnaire que dans le reste de la société). À partir de la variable tporgwk (type de travail), nous créons donc une variable indicatrice "public" (correspondant aux valeurs 1, 2 et 3 de tporgwk) et une autre "independant" (correpondant à la valeur 5 de tporgwk).

In [ ]:
base_indiv['public'] = np.where(base_indiv['tporgwk'].isin([1,2,3]), 1, 0)
base_indiv['independant'] = np.where(base_indiv['tporgwk']==5, 1, 0)

Avant de régresser les proxys sur les variables objectives, il reste à modifier légérement les autres variables indicatrices disponibles pour que la valeur 0 corresponde à la réponse "non" et la valeur 1 à la réponse "oui" (par défaut, 1 correspond à la valeur "oui" et 2 à la valeur "non").

In [ ]:
base_indiv[['rlgblg', 'blgetmg', 'crmvct', 'chldhm', 'uemp3m']] = base_indiv[['rlgblg', 'blgetmg', 'crmvct', 'chldhm', 'uemp3m']].apply(lambda x: x.replace(to_replace = [1,2], value = [1,0]), axis = 0)

On se restreint finalement aux observations ne contenant pas de valeurs manquantes pour pouvoir effectuer nos régressions (pour les deux seules variables que nous n'avions pas encore traité du fait qu'elles avaient un nombre non négligeables de valeurs manquantes).

In [ ]:
base_indiv_bis = base_indiv.dropna(subset = ['tporgwk', 'hinctnta']) 

### Proxy confiance

Nous commençons par régresser le niveau de confiance sur ce sous-ensemble de variables objectives.

In [ ]:
Y = base_indiv_bis['confiance']
X = base_indiv_bis[['rlgblg', 'blgetmg', 'crmvct', 'gndr', 'agea', 'chldhm', 'level_education', 'uemp3m', 'public', 'independant', 'hinctnta']]
X = sm.tools.add_constant(X)
mod = st.OLS(Y,X)
print(mod.fit().summary())

La qualité du fit est raisonnable (R2 = 0.13) pour une variable subjective. De plus, les signes des coefficients sont globalement cohérents, sauf peut-être pour la variable chldhm. De plus, contrairement à l'intuition évoquée plus haut, le type d'organisation pour lequel les individus travaillent ne semble pas jouer de rôle. Il est enfin étonnant de voir que l'appartenance à une religion ou à une minorité ethnique augmente le degré général de confiance. Si le message religieux peut inciter à faire plus confiance à son prochain, cela ne semble pas être une explication suffisante (si on régresse d'ailleurs la seule variable ppltrst, le coefficient associé à rlgblg devient d'ailleurs non significatif donc cela semble venir d'une plus grande confiance dans les hommes politiques, a priori difficelement explicable).

### Proxy bien-être

In [ ]:
Y = base_indiv_bis['bienêtre']
X = base_indiv_bis[['rlgblg', 'blgetmg', 'crmvct', 'gndr', 'agea', 'chldhm', 'level_education', 'uemp3m', 'public', 'independant', 'hinctnta']]
X = sm.tools.add_constant(X)
mod = st.OLS(Y,X)
print(mod.fit().summary())

La qualité du fit est similaire à celle du dégré de confiance (R2 = 0.11). Le signe et la significativité des coefficients sont ici encore plus attendus que précédemment, à l'exception peut-être de l'appartenance à une minorité ethnique dont le signe est encore une fois positif alors qu'on pourrait s'attendre à ce qu'affronter de potentielles discriminations ou autre difficulté spécifiques (communication avec les autres plus difficile, ...) conduisent à une baisse de bien-être. 

### Proxy protection

In [ ]:
Y = base_indiv_bis['protection']
X = base_indiv_bis[['rlgblg', 'blgetmg', 'crmvct', 'gndr', 'agea', 'chldhm', 'level_education', 'uemp3m', 'public', 'independant', 'hinctnta']]
X = sm.tools.add_constant(X)
mod = st.OLS(Y,X)
print(mod.fit().summary())

La qualité du fit est encore une fois similaire aux régressions précédentes (R2 = 0.11). On retrouve des résultats proches des régressions précédentes, à la différence qu'ici l'appartenance à une religion influe négativement sur le sentiment de sécurité (ou dit autrement positivement sur la demande de protection, car rappelons qu'une valeur haute du proxy protection signifie une faible demande de protection). Cela peut s'expliquer par le fait que l'appartenance à une religion est souvent associé à un certain conservatisme, dont l'une des caractéristiques est la demande de sécurité.

### Proxy progressisme

In [ ]:
Y = base_indiv_bis['progressisme']
X = base_indiv_bis[['rlgblg', 'blgetmg', 'crmvct', 'gndr', 'agea', 'chldhm', 'level_education', 'uemp3m', 'public', 'independant', 'hinctnta']]
X = sm.tools.add_constant(X)
mod = st.OLS(Y,X)
print(mod.fit().summary())

C'est finalement, presque malheureusement, pour la variable la moins pertinente pour expliquer le vote populiste que le fit avec les variables objectives est le meilleur (R2 = 0.22). On peut noter qu'ici seules les variables de niveau d'éducation et de revenu influent positivement cette variable, alors que les variables liées à une plus forte propension au conservatisme (l'appartenance à une religion ou à une minorité ethnique et l'âge) influent négativement sur le niveau de progressisme sociétal.

Pour conclure sur ces variables objectives individuelles, on peut remarquer que seules le niveau d'éducation et le niveau de vie impactent systématiquement positivement les variables qui impactent négativement le vote populiste. Bien que d'autres variables soient ponctuellement sources d'explications complémentaires, notre analyse tend donc à confirmer, voir à réactualiser dans la nouvelle ère politique qu'ouvre la montée du populisme, l'importance du capital culturel et économique (pour reprendre une typologie bouridieusienne) dans le choix électoral.

## Troisième partie : déterminants régionaux du ressenti individuel

Peut-on néanmoins approfondir encore l'analyse, en ajoutant aux variables objectives individuelles des variables objectives régionales ? Nous nous inspirons ici des travaux de l'analyste politique Jerôme Fourquet, exposés dans son ouvrage *L'archipel français*. L'idée de Jerôme Fourquet est qu'en plus de la situation individuelle de chacun